In [ ]:
# import
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# create function for evaluation
def metrics(true, pred):
    #true = true.dropna()
    #pred = pred.dropna()
    
    cm = confusion_matrix(true, pred, labels=[0,1])
    print(cm)
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]

    precision = float(tp/(tp+fp)) 
    recall = float(tp/(tp+fn))  
    f1 = float((2*precision*recall)/(precision+recall))
    tss = float((tp/(tp+fn)) - (fp/(fp+tn)))
    
    return precision, recall, f1, tss, tn, fp, fn, tp

In [ ]:
# settings - model
model_type = "autoreg60"

In [ ]:
# settings - CoV
var_coef_true_values = [0.5]
if "autoreg" in model_type:
    var_coef_pred_values = np.arange(0.001,0.0201, 0.0001) # for autoreg model
    x_axis = np.arange(0.000,0.0201,0.002)
else:
    var_coef_pred_values = np.arange(0.01,0.201, 0.001) # for multiNN model
    x_axis = np.arange(0.00,0.201, 0.02)

In [ ]:
# data loading
df = pd.read_csv(f"post_processing/{model_type}_predictions_on_test-data_with_time-bin_std-cov_full-test.csv")
df

In [ ]:
# evaluate
results = pd.DataFrame(columns=["cov_true", "cov_test", "precision", "recall", "f1", "tss", "tn", "fp", "fn", "tp"])
for vc_true in var_coef_true_values:
    df["True"] = np.where(df[f"{model_type}_cov_test"] < vc_true, 0, 1)
    for vc_pred in var_coef_pred_values:
        df["Pred"] = np.where(df[f"{model_type}_cov_pred"] < vc_pred, 0, 1)
        
        true_col = 'True'
        pred_col = 'Pred'
        
        # vyhodnotenie metrik
        print("Vyhodnotenie pre hodnoty variačných koeficientov: pred %g, true %3.1f \n" %(vc_pred, vc_true))
        res = metrics(df[true_col], df[pred_col])
        results.loc[len(results)] = [vc_true, vc_pred, res[0], res[1], res[2], res[3], res[4], res[5], res[6], res[7]]

In [ ]:
results

In [ ]:
# save results
results.to_csv(f"evaluation/{model_type}.csv")

In [ ]:
# plot results
plt.figure(figsize=(8,4))
plt.scatter(results["cov_test"], results["precision"],s=1,label="precision")
plt.scatter(results["cov_test"], results["recall"],s=1,label="recall")
plt.scatter(results["cov_test"], results["tss"],s=1,label="TSS")
plt.xlabel("Coefficient of variation - predicted values", fontsize=13)
plt.ylabel("Metric value", fontsize=13)
plt.title(f"Metrics for different CoV of prediction settings, {model_type} model", fontsize=14)
plt.legend(markerscale=3)
plt.grid()
plt.tight_layout()
plt.xticks(x_axis, size=10)
plt.savefig(f"evaluation/{model_type}.png")
plt.show()